In [1]:
!pip install rouge_score
!pip install trl
!pip install bert-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=869daca3504db69aaa53bf9df73896e94e728321604551823f319b9eb857efec
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's 

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from rouge_score import rouge_scorer
from bert_score import score as bert_score
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from tqdm import tqdm
import pandas as pd
import numpy as np
import torch
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer
from datasets import Dataset

In [4]:
# Define the model and tokenizer
LM_modelname = "meta-llama/Meta-Llama-3-8B"  # Base model name


In [5]:
# Load base model and tokenizer
base_model = AutoModelForCausalLM.from_pretrained(LM_modelname)
base_tokenizer = AutoTokenizer.from_pretrained(LM_modelname)

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [6]:
# Add padding token and resize embeddings
base_tokenizer.add_special_tokens({'pad_token': '[PAD]'})
base_model.resize_token_embeddings(len(base_tokenizer))

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(128257, 4096)

In [8]:

# Load the evaluation datasets
fdic_dataset_path = '/content/fdic_terms_definitions.csv'
abbreviation_dataset_path = '/content/Task1-Abbreviation-acronym-validation.csv'

fdic_dataset = pd.read_csv(fdic_dataset_path, encoding='latin-1')
abbreviation_dataset = pd.read_csv(abbreviation_dataset_path, encoding='latin-1')


In [9]:
# Extract questions and references for FDIC dataset
fdic_questions = fdic_dataset['Term'].tolist()
fdic_references = fdic_dataset['Definition'].tolist()

In [16]:
# Extract abbreviations and full forms for Abbreviation dataset
abbr_questions = abbreviation_dataset['abbreviations'].tolist()
abbr_references = abbreviation_dataset['answer'].tolist()

In [15]:
# Define maximum length for generation
max_length = 200


In [17]:
# Store results
fdic_predictions = []
abbr_predictions = []

In [18]:
# Evaluate on FDIC Dataset
print("Generating predictions for FDIC Dataset...")
for question in tqdm(fdic_questions, desc="FDIC Evaluation"):
    inputs = base_tokenizer.encode_plus(question, return_tensors="pt", padding=True, truncation=True)
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]

    if base_model.config.pad_token_id is None:
        base_model.config.pad_token_id = base_model.config.eos_token_id

    output_ids = base_model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=max_length,
        num_return_sequences=1
    )
    prediction = base_tokenizer.decode(output_ids[0], skip_special_tokens=True)
    fdic_predictions.append(prediction.strip())


Generating predictions for FDIC Dataset...


FDIC Evaluation:   0%|          | 0/47 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
FDIC Evaluation: 100%|██████████| 47/47 [1:16:21<00:00, 97.48s/it] 


In [ ]:
# Evaluate on Abbreviation Dataset
print("Generating predictions for Abbreviation Dataset...")
for question in tqdm(abbr_questions, desc="Abbreviation Evaluation"):
    inputs = base_tokenizer.encode_plus(question, return_tensors="pt", padding=True, truncation=True)
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]

    if base_model.config.pad_token_id is None:
        base_model.config.pad_token_id = base_model.config.eos_token_id

    output_ids = base_model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=max_length,
        num_return_sequences=1
    )
    prediction = base_tokenizer.decode(output_ids[0], skip_special_tokens=True)
    abbr_predictions.append(prediction.strip())

In [19]:
# Calculate BERT Scores for FDIC Dataset
print("Calculating BERT Scores for FDIC Dataset...")
P_fdic, R_fdic, F1_fdic = bert_score(fdic_predictions, fdic_references, lang="en", verbose=True)

Calculating BERT Scores for FDIC Dataset...


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/2 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 1.92 seconds, 24.52 sentences/sec


In [23]:
# Print Results
print("\nFDIC Dataset BERT Scores:")
print(f"Precision: {P_fdic.mean().item():.4f}, Recall: {R_fdic.mean().item():.4f}, F1: {F1_fdic.mean().item():.4f}")

#print("\nAbbreviation Dataset BERT Scores:")
#print(f"Precision: {P_abbr.mean().item():.4f}, Recall: {R_abbr.mean().item():.4f}, F1: {F1_abbr.mean().item():.4f}")



FDIC Dataset BERT Scores:
Precision: 0.7983, Recall: 0.8220, F1: 0.8097


In [ ]:
# Display Sample Predictions for FDIC Dataset
print("\nSample FDIC Predictions:")
num_samples = 5  # Number of examples to display
for i in range(min(num_samples, len(fdic_questions))):
    print(f"\nExample {i + 1}:")
    print(f"Question (Term): {fdic_questions[i]}")
    print(f"Generated Prediction: {fdic_predictions[i]}")
    print(f"Reference (Ground Truth): {fdic_references[i]}")


In [ ]:
# Display Sample Predictions for Abbreviation Dataset
print("\nSample Abbreviation Predictions:")
for i in range(min(num_samples, len(abbr_questions))):
    print(f"\nExample {i + 1}:")
    print(f"Abbreviation: {abbr_questions[i]}")
    print(f"Generated Prediction: {abbr_predictions[i]}")
    print(f"Reference (Ground Truth): {abbr_references[i]}")